In [1]:
from langchain.document_loaders import TextLoader, PyPDFLoader
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
import numpy as np
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.vectorstores import Milvus
import os
from langchain.document_loaders import TextLoader, PyPDFLoader

# document_embeddings

In [2]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader

# Folder tempat dokumen berada
DATA_FOLDER = "data/"

# Fungsi untuk membaca dokumen
def load_documents(folder_path):
    documents = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if file_name.endswith(".txt"):
            loader = TextLoader(file_path)
        elif file_name.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        else:
            print(f"Unsupported file format: {file_name}")
            continue
        
        # Load documents from the file and extend the list
        doc_texts = loader.load()  # loader.load() returns a list of documents (probably with text as the attribute)
        
        # Extract text from documents if the loader returns objects
        for doc in doc_texts:
            documents.append(doc.page_content)  # Assuming 'page_content' holds the text data for each document
            
    return documents

# Test membaca dokumen
docs = load_documents(DATA_FOLDER)
print(f"Loaded {len(docs)} documents.")

# Fungsi untuk preprocess dan membersihkan teks
def preprocess_text(documents):
    # Contoh preprocessing: mengganti newline dengan spasi dan mengubah semua teks ke lowercase
    return [doc.replace("\n", " ").lower() for doc in documents]

# Preprocess dan bersihkan teks
cleaned_documents = preprocess_text(docs)

# Tampilkan dokumen pertama setelah diproses
print(cleaned_documents[0])  # Print first cleaned document


Unsupported file format: .ipynb_checkpoints
Loaded 1 documents.
nama saya fikri rama saya kerja sebagai data analyst umur saya 27 saya bingung harus apa teman saya ada 5 orang


In [59]:
# # Inisialisasi text splitter
# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# # Potong dokumen menjadi chunk
# chunked_docs = []
# for doc in docs:
#     chunks = text_splitter.split_text(doc.page_content)
#     chunked_docs.extend(chunks)

# # Inisialisasi model embedding (menggunakan nomic-embed-text)
# embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# # Buat embedding untuk setiap chunk dokumen
# embeddings = [embedding_model.embed_query(chunk) for chunk in chunked_docs]

In [3]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Milvus

# Load the embedding model
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# Generate embeddings
def create_embeddings(documents):
    embeddings = [embedding_model.embed_query(chunk) for chunk in documents]
    return embeddings

embeddings = create_embeddings(cleaned_documents)
# print(embeddings[0])  # Print first embedding

C:\Users\User\AppData\Local\Temp\ipykernel_11732\1488634427.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="nomic-embed-text")


In [29]:
# print(embeddings[0])  # Print first embedding

In [4]:
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType
# Koneksi ke Milvus server
connections.connect(host="192.168.30.222", port="19530")
# Definisi schema untuk koleksi (Milvus)
def create_milvus_collection():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768),  # Adjust dim based on your model
        FieldSchema(name="document", dtype=DataType.VARCHAR, max_length=500)
    ]
    schema = CollectionSchema(fields)
    collection = Collection(name="document_embeddings", schema=schema)
    return collection

collection = create_milvus_collection()

# Buat indeks
collection.create_index(
    field_name="embedding",
    index_params={"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}}
)
# Muat koleksi dan indeks ke memori
collection.load()

def insert_embeddings_into_milvus(collection, embeddings, documents):
    collection.insert([embeddings, documents])

insert_embeddings_into_milvus(collection, embeddings, cleaned_documents)


In [5]:
# Buat indeks
collection.create_index(
    field_name="embedding",
    index_params={"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}}
)
# Muat koleksi dan indeks ke memori
collection.load()

def insert_embeddings_into_milvus(collection, embeddings, documents):
    collection.insert([embeddings, documents])

insert_embeddings_into_milvus(collection, embeddings, cleaned_documents)

In [6]:
collection.flush()

In [7]:
from pymilvus import utility

# # Query
search_vector = embeddings[0]  # Contoh menggunakan embedding pertama
results = collection.search(
    data=[search_vector],
    anns_field="embedding",
    param={"metric_type": "L2", "params": {"nprobe": 10}},
    limit=5,
)

# # Tampilkan hasil pencarian
if results:
    for result in results[0]:
        print(f"Match ID: {result.id}, Distance: {result.distance}")
else:
    print("No matching results found.")



Match ID: 454238142078124776, Distance: 0.0
Match ID: 454238142078124778, Distance: 0.0
Match ID: 454238142078124780, Distance: 0.0
Match ID: 454238142078124782, Distance: 0.0


In [8]:
from pymilvus import connections

# Koneksi ke Milvus menggunakan IP dan port
milvus_host = "192.168.30.222"  # IP Milvus server
milvus_port = "19530"           # Port Milvus server

connections.connect(alias="default", host=milvus_host, port=milvus_port)
print(connections)

In [9]:
from pymilvus import connections
# print(connections.list_connections())
# print(connections.get_connection(alias="default").server_status())
print(len(embeddings[0]))  # Output ini harus sama dengan "dim" di Milvus schema
# connections.disconnect("default")

768


In [10]:
from langchain.schema import Document  # Import the Document class
from langchain.vectorstores import Milvus

# Setup Llama3 model
# llm = Ollama(model="llama3")  # Adjust as per your model setup

# Convert cleaned documents (strings) into Document objects
documents = [Document(page_content=doc) for doc in cleaned_documents]


# Buat retriever dari Milvus
retriever = Milvus.from_documents(
    documents = documents,
    collection_name="document_embeddings",
    embedding=embeddings,
    connection_args={"uri":"http://192.168.30.222:19530"}  # Pastikan alias koneksi yang benar digunakan
)

# "uri":"http://192.168.30.222:19530", 
#     token="root:root123"

RPC error: [create_index], <MilvusException: (code=1, message=cannot create index on non-existed field: vector)>, <Time:{'RPC start': '2024-11-28 18:12:01.997124', 'RPC error': '2024-11-28 18:12:02.001293'}>
RPC error: [create_index], <MilvusException: (code=1, message=cannot create index on non-existed field: vector)>, <Time:{'RPC start': '2024-11-28 18:12:02.003146', 'RPC error': '2024-11-28 18:12:02.008127'}>
Failed to create an index on collection: document_embeddings


MilvusException: <MilvusException: (code=1, message=cannot create index on non-existed field: vector)>

In [13]:
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.schema import Document  # Import the Document class
from langchain.vectorstores import Milvus

# Setup Llama3 model
llm = Ollama(model="llama3")  # Adjust as per your model setup

# Convert cleaned documents (strings) into Document objects
documents = [Document(page_content=doc) for doc in cleaned_documents]

# Setup LangChain for RAG
# Connect to Milvus server at the remote address
milvus_host = "192.168.30.222"  # The IP of the server where Milvus is running
milvus_port = "19530"  # Default port for Milvus

# Make the connection
connections.connect("default", host=milvus_host, port=milvus_port)
# connections.connect("default", host="192.168.30.222", port="19530")


# Setup LangChain for RAG
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" method, combine retrieval and generation
    retriever=Milvus.from_documents(documents,
                                    embedding=embeddings,
                                    connection_args={"uri":"http://192.168.30.222:19530"})  # Pass documents to the retriever
)

# Test the retrieval and generation
query = "What is the main topic of the document?"
response = retrieval_qa_chain.run(query)
print(response)

AttributeError: 'list' object has no attribute 'embed_documents'

In [65]:
# from langchain.vectorstores import Milvus
# from langchain.embeddings import OpenAIEmbeddings

# # Konfigurasi koneksi ke Milvus
# milvus_connection_args = {
#     "host": "192.168.30.222",  # Ganti dengan host Milvus jika berbeda
#     "port": "19530"       # Port default Milvus
# }

# # Inisialisasi vector store menggunakan koleksi yang sudah ada
# vector_store = Milvus(
#     embedding_function=None,  # Tidak diperlukan karena sudah ada embedding
#     collection_name='document_embeddings',
#     connection_args=milvus_connection_args,
# )
# # print(f"Connected to Milvus collection: {collection_name}")


In [66]:
# # Query retrieval menggunakan LangChain
# query = "siapa fikri rama?"  # Ganti dengan query yang relevan
# query_embedding = embedding_model.embed_query(query)  # Embed query

# # Pencarian menggunakan LangChain-Milvus
# results = vector_store.similarity_search_by_vector(
#     query_embedding, k=5  # Jumlah hasil yang ingin diambil
# )

# # Tampilkan hasil
# if results:
#     print("Retrieved Documents:")
#     for idx, result in enumerate(results, start=1):
#         print(f"{idx}. {result.page_content} (Score: {result.metadata})")
# else:
#     print("No relevant documents found.")


In [67]:
# from langchain.chains import RetrievalQA
# from langchain.llms import Ollama
# from langchain.schema import Document
# from langchain.vectorstores import Milvus
# from langchain.embeddings import OpenAIEmbeddings
# from pymilvus import connections

# # Configuration for Milvus connection
# milvus_connection_args = {
#     "host": "192.168.30.222",  # Replace with your Milvus server host
#     "port": "19530"            # Default port for Milvus
# }

# # Connect to Milvus server
# connections.connect("default", host=milvus_connection_args['host'], port=milvus_connection_args['port'])

# # Initialize OpenAI embeddings
# embeddings = embeddings

# # Convert cleaned documents (strings) into Document objects
# documents = [Document(page_content=doc) for doc in cleaned_documents]

# # Initialize Milvus vector store with the existing collection name
# vector_store = Milvus(
#     embedding_function=embeddings,  # Using the OpenAI Embeddings
#     collection_name='document_embeddings',  # Use your collection name here
#     connection_args=milvus_connection_args  # Connection details for Milvus
# )

# print(f"Connected to Milvus collection: document_embeddings")

# # Setup LangChain for RAG (Retrieval-Augmented Generation)
# retrieval_qa_chain = RetrievalQA.from_chain_type(
#     llm=Ollama(model="llama3"),  # Setup Llama3 model
#     chain_type="stuff",  # "stuff" method, combines retrieval and generation
#     retriever=vector_store.as_retriever()  # Using the Milvus vector store as retriever
# )

# # Test the retrieval and generation with a sample query
# query = "What is the main topic of the document?"
# response = retrieval_qa_chain.run(query)

# # Print the result of the query
# print(response)


In [ ]:
from pymilvus import Collection
collection = Collection("document_embeddings")      # Get an existing collection.
collection.load()

In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.schema import Document
from langchain.vectorstores import Milvus
from langchain.embeddings import OpenAIEmbeddings
from pymilvus import connections

# Setup Llama3 model
llm = Ollama(model="llama3")  # Adjust as per your model setup

# Initialize OpenAI embeddings
embeddings = [embedding_model.embed_query(chunk) for chunk in documents]

# Convert cleaned documents (strings) into Document objects
documents = [Document(page_content=doc) for doc in cleaned_documents]

# Setup Milvus connection
milvus_host = "192.168.30.222"  # The IP of the server where Milvus is running
milvus_port = "19530"  # Default port for Milvus

# Connect to Milvus server
connections.connect("default", host=milvus_host, port=milvus_port)

# Embed the documents
# document_embeddings = embeddings.embed_documents([doc.page_content for doc in documents])

# Set up the Milvus vector store
vector_store = Milvus(
    embedding_function=embeddings,  # Set the embedding function
    collection_name="document_embeddings",  # Your Milvus collection name
    connection_args={"host": milvus_host, "port": milvus_port},  # Connection details
)

# Insert the documents and embeddings into Milvus
vector_store.add_texts([doc.page_content for doc in documents], embeddings=document_embeddings)

# Setup LangChain for RAG (Retrieval-Augmented Generation)
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" method, combines retrieval and generation
    retriever=vector_store.as_retriever()  # Use the Milvus vector store as retriever
)

# Test the retrieval and generation with a sample query
query = "What is the main topic of the document?"
response = retrieval_qa_chain.run(query)

# Print the result of the query
print(response)


RPC error: [create_index], <MilvusException: (code=1, message=cannot create index on non-existed field: vector)>, <Time:{'RPC start': '2024-11-28 18:16:47.775138', 'RPC error': '2024-11-28 18:16:47.782607'}>
RPC error: [create_index], <MilvusException: (code=1, message=cannot create index on non-existed field: vector)>, <Time:{'RPC start': '2024-11-28 18:16:47.784605', 'RPC error': '2024-11-28 18:16:47.792629'}>
Failed to create an index on collection: document_embeddings


MilvusException: <MilvusException: (code=1, message=cannot create index on non-existed field: vector)>